## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data = pd.read_csv("WeatherPy_Database.csv")
city_data.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Aklavik,CA,68.2191,-135.0107,35.60,87,75,17.27,broken clouds
1,1,Kahului,US,20.8947,-156.4700,82.98,74,20,12.66,few clouds
2,2,Dikson,RU,73.5069,80.5464,29.84,96,100,18.88,light snow
3,3,Mataura,NZ,-46.1927,168.8643,39.15,96,7,1.90,clear sky
4,4,Port Elizabeth,ZA,-33.9180,25.5701,52.09,76,40,3.44,scattered clouds
5,5,Cabo San Lucas,MX,22.8909,-109.9124,91.81,50,1,4.61,clear sky
6,6,Kokopo,PG,-4.3520,152.2633,76.03,88,100,4.36,overcast clouds
7,7,Carahue,CL,-38.7000,-73.1667,48.40,68,100,1.36,overcast clouds
8,8,Kodiak,US,57.7900,-152.4072,47.71,76,1,6.91,clear sky
9,9,Lakatoro,VU,-16.0999,167.4164,74.53,76,100,7.34,overcast clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data.loc[(city_data["Max Temp"] <= max_temp) & \
                                       (city_data["Max Temp"] >= min_temp)]

In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                218
City                   218
Country                217
Lat                    218
Lng                    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Current Description    218
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kahului,US,82.98,few clouds,20.8947,-156.4700,
6,Kokopo,PG,76.03,overcast clouds,-4.3520,152.2633,
11,Atuona,PF,79.70,light rain,-9.8000,-139.0333,
12,Albany,US,80.74,overcast clouds,42.6001,-73.9662,
14,Eyl,SO,80.96,broken clouds,7.9803,49.8164,
15,Vallenar,CL,80.35,overcast clouds,-28.5708,-70.7581,
19,Salalah,OM,84.29,few clouds,17.0151,54.0924,
20,Trairi,BR,82.56,overcast clouds,-3.2778,-39.2689,
21,Neiafu,TO,75.88,few clouds,-18.6500,-173.9833,
24,Victoria,HK,86.38,broken clouds,22.2855,114.1577,


In [14]:
import numpy as np

In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
 
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))